In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame as DF
from sympy import diff
from sympy.abc import x,y
from sympy import cos
from sympy import *
from sympy import lambdify
from itertools import product
import random
from collections import defaultdict
import copy
import time

**Define path**

In [2]:
# Defining path to folder raw data is saved in
path = "C:/Users/robtk/OneDrive/Desktop/DIAS Internship/Raw data/Harmonic oscillator warm up/"

### Set display of pandas dataframes and Series to 11 decimal places

In [3]:
pd.set_option("display.precision", 11)

**Define the commutator**

In [4]:
def commutator(A,B):
    
    # matmul() multiplies two matrices together
    return np.matmul(A,B) - np.matmul(B,A)

In [5]:
M1 = np.array([[1,2,3], [4,5,6], [7,8,9]])
M2 = np.array([[8,7,6], [5,4,3], [3,2,1]])
M3 = np.array([[3,4,2],[5,6,7],[4,3,2]])
M4 = np.array([[2,1,2], [1,2,1], [2,1,2]])

In [6]:
commutator(M2, commutator(M1, M2))

array([[ 1566,   747,   -72],
       [  -39,  -354,  -669],
       [-1332, -1272, -1212]])

In [7]:
commutator(M3, commutator(M1, M3))

array([[ 267,  170,  -50],
       [ 774,  608,  157],
       [-630, -808, -875]])

In [8]:
commutator(M2, commutator(M1, M2)) + commutator(M3, commutator(M1, M3)) + commutator(M1, commutator(M1, M1)) + commutator(M4, commutator(M1, M4))

array([[ 1939,   961,   -64],
       [  743,   234,  -540],
       [-2000, -2144, -2173]])

### Define Lagrangian

In [9]:
def L(X_list, V_list, g):
    
    X_array = np.array(X_list)
    V_array = np.array(X_list)
    term_1 =  np.sum( np.trace( (V_array)**2 ) )
    
    # Calculate the sum of commutators found in the second term of the lagrangian
    for el in X_list:
        X_cut = [x for x in X_list if x != el]
        Sum2 = np.sum( np.trace( (commutator(el,X_cut))**2 ) ) 
    
    term_2 = 1/2 * Sum2
    
    return 1/(2*g**2) * (term_1 + term_2)

**Define the Hamiltonian**

In [10]:
def H(X_list, V_list, g):
    
    V_array = np.array(X_list)
    V_array_T = np.transpose(V_array)
    
    return sum( V_array_T * V_array /g**2) - L(X_list, V_list, g)
    

Initial reference point is x_0 = (X, V)

**Define the acceleration equation of motion as gotten from the research paper.**

In [11]:
def acceleration(X_list):
    
    # This function creates a list of all the accelerations of all the coordinates from a list of the coordinate matrices.
    
    # List to be filled with accelerations for the coordinates
    A_list = []
    
    # Turn the coordinates into an array so all X^j can put into the equation of motion in one line.
    array = np.array(X_list)
    
    # for loop to calculate all the coordinate accelerations.
    for el in X_list:
        
        # Doing the equation of motion
        # el == X^i and array == X^j in the equation of motion
        equation_of_motion = commutator(array ,commutator(el, array))
        
        # sum() carries out the summation symbol in the equation of motion and so the acceleration is calculated.
        # append the acceleration to the list.
        A_list.append(sum(equation_of_motion))
        
    return A_list

**Code for evolving the initial point in phase space for 1000 seconds so that the system is thermalised.**

In [18]:
def thermalising_algorithm(X_list, V_list, A_list, delta_t):
    
    # Define initial conditions as numpy arrays
    X_array_old = np.array(X_list)
    V_array_old = np.array(V_list)
    A_array_old = np.array(A_list)
    
    i = 0
    # Repeating algorithm for 1000 seconds to thermalise the system.
    
    while i < 100/delta_t:
        
        # b) velocity Verlet 1 to get new positions from old positions, momentums and rate of change of momentums
        
        X_array_new = (X_array_old + V_array_old * delta_t + 1/2 * A_array_old * delta_t**2)

        
        # c) Use equations of motion to get new acceleration values
        
        A_array_new = np.array(acceleration(X_array_new.tolist()))

        # d) Use Velocity Verlet 2 to get new momentums

        V_array_new = (V_array_old + (1/2) * (A_array_new + A_array_old) * delta_t)
                
        # Update coordinates
        X_array_old = X_array_new
        V_array_old = V_array_new
        A_array_old = A_array_new
        
        if i % 1000 == True:
            print("Thermal", i, X_array_new)

        
        i += 1
        
    return X_array_new, V_array_new, A_array_new

**Code for evolving the simulation and recording the position and momentum after it was thermalised.**

In [45]:
def evolving_algorithm(X_list, V_list, A_list, delta_t, record_steps, simulation_repetitions, 
                       X_labels, V_labels, N, times):
    
    i = 1
    
    # Define initial conditions as numpy arrays
    X_array_old = np.array(X_list)
    V_array_old = np.array(V_list)
    A_array_old = np.array(A_list)
    
    # Coordinates dictionaries
    recorded_X1 = { "0" : X_list[0].tolist()}
    recorded_X2 = { "0" : X_list[1].tolist()}
    recorded_V1 = { "0" : V_list[0].tolist()}
    recorded_V2 = { "0" : V_list[1].tolist()}
    recorded_X3 = { "0" : X_list[2].tolist()}
    recorded_X4 = { "0" : X_list[3].tolist()}
    recorded_V3 = { "0" : V_list[2].tolist()}
    recorded_V4 = { "0" : V_list[3].tolist()}
    recorded_X5 = { "0" : X_list[4].tolist()}
    recorded_X6 = { "0" : X_list[5].tolist()}
    recorded_V5 = { "0" : V_list[4].tolist()}
    recorded_V6 = { "0" : V_list[5].tolist()}
    recorded_X7 = { "0" : X_list[6].tolist()}
    recorded_X8 = { "0" : X_list[7].tolist()}
    recorded_V7 = { "0" : V_list[6].tolist()}
    recorded_V8 = { "0" : V_list[7].tolist()}
    recorded_X9 = { "0" : X_list[8].tolist()}
    recorded_V9 = { "0" : X_list[8].tolist()}



    # Repeating algorithm
    while i < simulation_repetitions:
        
        # b) velocity Verlet 1 to get new positions from old positions, momentums and rate of change of momentums
        
        X_array_new = (X_array_old + V_array_old * delta_t + 1/2 * A_array_old * delta_t**2)

        
        # c) Use equations of motion to get new acceleration values
        
        A_array_new = np.array(acceleration(X_array_new.tolist()))


        # d) Use Velocity Verlet 2 to get new momentums

        V_array_new = (V_array_old + (1/2) * (A_array_new + A_array_old) * delta_t)
        
        
        # Recording position, momentum, and rate of change of momentum every 10th step
        if i % record_steps == 0:

            recorded_X1[f"{delta_t*record_steps*i}"] = X_array_new[0].tolist()
            recorded_X2[f"{delta_t*record_steps*i}"] = X_array_new[1].tolist()
            recorded_V1[f"{delta_t*record_steps*i}"] = V_array_new[0].tolist()
            recorded_V2[f"{delta_t*record_steps*i}"] = V_array_new[1].tolist()
            
            recorded_X3[f"{delta_t*record_steps*i}"] = X_array_new[2].tolist()
            recorded_X4[f"{delta_t*record_steps*i}"] = X_array_new[3].tolist()
            recorded_V3[f"{delta_t*record_steps*i}"] = V_array_new[2].tolist()
            recorded_V4[f"{delta_t*record_steps*i}"] = V_array_new[3].tolist()
            
            recorded_X5[f"{delta_t*record_steps*i}"] = X_array_new[4].tolist()
            recorded_X6[f"{delta_t*record_steps*i}"] = X_array_new[5].tolist()
            recorded_V5[f"{delta_t*record_steps*i}"] = V_array_new[4].tolist()
            recorded_V6[f"{delta_t*record_steps*i}"] = V_array_new[5].tolist()
            
            recorded_X7[f"{delta_t*record_steps*i}"] = X_array_new[6].tolist()
            recorded_X8[f"{delta_t*record_steps*i}"] = X_array_new[7].tolist()
            recorded_V7[f"{delta_t*record_steps*i}"] = V_array_new[6].tolist()
            recorded_V8[f"{delta_t*record_steps*i}"] = V_array_new[7].tolist()
            
            recorded_X9[f"{delta_t*record_steps*i}"] = X_array_new[8].tolist()
            recorded_V9[f"{delta_t*record_steps*i}"] = V_array_new[8].tolist()

            
            print(X_array_new)

        
            
        # Update coordinates
        X_array_old = X_array_new
        V_array_old = V_array_new
        A_array_old = A_array_new
         

        i += 1
    
    return recorded_X1, recorded_X2, recorded_V1, recorded_V2, recorded_X3, recorded_X4, recorded_V3, recorded_V4, recorded_X5, recorded_X6, recorded_V5, recorded_V6, recorded_X7, recorded_X8, recorded_V7, recorded_V8, recorded_X9, recorded_V9


**Create initial position matrices: Take X^i to have random (Gaussian) matrix elements, and be traceless.**

In [46]:
def simulation(dim, N, delta_t):
    # N is the length of the side of the individual position square matrices.
    # dim is the number of position coordinates we need.
    
    # Create position coordinate matrices with gaussian distributed elements.
    
    X_list = np.random.normal(0, 1, size = (dim, N, N))

        
    # The velocity Verlet algorithm conserves Gauss' law, so if it starts off true, it remains true. 
    # So just ensure the initial V^i are zero matrices.  

    V_initial = np.zeros((N, N))
    i = 0
    V_list = []
    while i < dim:
        V_list.append(V_initial)
        i += 1
    print(V_list)
    # Use the scalar equation of motion in our gauge to calculate the initial accelerations from the initial positions
    
    A_list = acceleration(X_list)
    

    # Define number of steps between measurement recordings
    record_steps = 100
    
    # For checking if H is conserved
    #print(H())

    
    # How many iterations the simulation will go on for after it has been thermalised.
    simulation_repetitions = 1000000
    # Thermalise the system. , i.e. evolve it for long enough time that it presumably reaches a "typical configuration".
    # Here Swapno suggests to evolve it, starting anywhere, for 1000 seconds, dt = 10e-5 s.
 
    thermalised_X_array, thermalised_V_array, thermalised_A_array = thermalising_algorithm(X_list, V_list, A_list, delta_t)
        
    # Labels for coordinates dictionary
    X_labels = []
    j = 1
    while j <= dim:
        X_labels.append(f"X {j}")
        
        j+=1
    
    V_labels = []
    j = 1
    while j <= dim:
        V_labels.append(f"V {j}")
        
        j+=1
    
    
    # Define the times after thermalisation that the coordinates were recorded at.
    times = np.arange(0, simulation_repetitions*delta_t + delta_t*record_steps, delta_t*record_steps)
    
    # Now that the system is thermalised and the thermalised initial conditions are found, evolve the system
    # and run the data collecting simulation.
    recorded_X1, recorded_X2, recorded_V1, recorded_V2 = evolving_algorithm(thermalised_X_array,
                                                                            thermalised_V_array, 
                                                                            thermalised_A_array,
                                          delta_t, record_steps, simulation_repetitions, 
                                          X_labels, V_labels, N, times)
    
    # Make dataframes out of the dictionaries
    
    # Check if H is conserved

    X1 = pd.Series(recorded_X1).T
    X2 = pd.Series(recorded_X2).T
    V1 = pd.Series(recorded_V1).T
    V2 = pd.Series(recorded_V2).T

    X = pd.concat([X1, X2], axis = 1)
    X = X.rename(columns = {0:"X 1", 1:"X 2"})
    
    V = pd.concat([V1, V2], axis = 1)
    V = V.rename(columns = {0:"V 1", 1:"V 2"})
    

    
    return X, V

In [53]:
def simulation_no_thermalising(dim, N, delta_t):
    # N is the length of the side of the individual position square matrices.
    # dim is the number of position coordinates we need.
    
    # Create position coordinate matrices with gaussian distributed elements.
    
    X_list = np.random.normal(0, 1, size = (dim, N, N))

        
    # The velocity Verlet algorithm conserves Gauss' law, so if it starts off true, it remains true. 
    # So just ensure the initial V^i are zero matrices.  

    V_initial = np.zeros((N, N))
    i = 0
    V_list = []
    while i < dim:
        V_list.append(V_initial)
        i += 1
    print(V_list)
    # Use the scalar equation of motion in our gauge to calculate the initial accelerations from the initial positions
    
    A_list = acceleration(X_list)
    

    # Define number of steps between measurement recordings
    record_steps = 100
    
    # For checking if H is conserved
    #print(H())

    
    # How many iterations the simulation will go on for after it has been thermalised.
    simulation_repetitions = 200000
    # Thermalise the system. , i.e. evolve it for long enough time that it presumably reaches a "typical configuration".
    # Here Swapno suggests to evolve it, starting anywhere, for 1000 seconds, dt = 10e-5 s.

    # Labels for coordinates dictionary
    X_labels = []
    j = 1
    while j <= dim:
        X_labels.append(f"X {j}")
        
        j+=1
    
    V_labels = []
    j = 1
    while j <= dim:
        V_labels.append(f"V {j}")
        
        j+=1
    
    
    # Define the times after thermalisation that the coordinates were recorded at.
    times = np.arange(0, simulation_repetitions*delta_t + delta_t*record_steps, delta_t*record_steps)
    
    # Now that the system is thermalised and the thermalised initial conditions are found, evolve the system
    # and run the data collecting simulation.
    recorded_X1, recorded_X2, recorded_V1, recorded_V2, recorded_X3, recorded_X4, recorded_V3, recorded_V4, recorded_X5, recorded_X6, recorded_V5, recorded_V6, recorded_X7, recorded_X8, recorded_V7, recorded_V8, recorded_X9, recorded_V9 = evolving_algorithm(X_list,
                                                                            V_list, 
                                                                            A_list,
                                          delta_t, record_steps, simulation_repetitions, 
                                          X_labels, V_labels, N, times)
    
    # Make dataframes out of the dictionaries
    
    # Check if H is conserved

    X1 = pd.Series(recorded_X1).T
    X2 = pd.Series(recorded_X2).T
    V1 = pd.Series(recorded_V1).T
    V2 = pd.Series(recorded_V2).T
    
    X3 = pd.Series(recorded_X3).T
    X4 = pd.Series(recorded_X4).T
    V3 = pd.Series(recorded_V3).T
    V4 = pd.Series(recorded_V4).T
    
    X5 = pd.Series(recorded_X5).T
    X6 = pd.Series(recorded_X6).T
    V5 = pd.Series(recorded_V5).T
    V6 = pd.Series(recorded_V6).T
    
    X7 = pd.Series(recorded_X7).T
    X8 = pd.Series(recorded_X8).T
    V7 = pd.Series(recorded_V7).T
    V8 = pd.Series(recorded_V8).T
    
    X9 = pd.Series(recorded_X9).T
    V9 = pd.Series(recorded_V9).T


    X = pd.concat([X1, X2, X3, X4, X5, X6, X7, X8, X9], axis = 1)
    X = X.rename(columns = {0:"X 1", 1:"X 2", 2 : "X3", 3 : "X4", 4 : "X5", 5 : "X6", 6 : "X7", 7 : "X8", 8: "X9"})
    
    V = pd.concat([V1, V2,  X3, X4, X5, X6, X7, X8, X9], axis = 1)
    V = V.rename(columns = {0:"V 1", 1:"V 2", 2 : "V 3", 3 : "V 4", 4 : "V 5", 5 : "V 6", 6 : "V 7", 7 : "V 8" , 8 : "V 9" })
    

    
    return X, V

In [54]:
start = time.time()
X, V= simulation_no_thermalising(9, 2, 10**-4)

print(time.time()-start)

[array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]])]
[[[-0.50232023 -0.52106332]
  [ 0.68751     0.84631551]]

 [[ 1.13283438 -0.31057958]
  [ 1.74444286 -0.60848369]]

 [[ 1.90105267 -0.76235203]
  [ 1.83833108  2.76631814]]

 [[ 1.51085459 -1.13033097]
  [ 0.42657414 -0.05901442]]

 [[ 1.12902334 -1.50653463]
  [-1.80715438 -1.17335889]]

 [[ 0.32757665 -0.5733492 ]
  [-0.9121043  -1.13570393]]

 [[ 0.83604441  0.29253576]
  [-0.62428108 -0.22336229]]

 [[-0.43286936 -0.46065251]
  [ 0.62638275  2.37213906]]

 [[-1.98671616 -0.8088073 ]
  [-0.42230164  0.41137997]]]
[[[-0.50191741 -0.51710656]
  [ 0.6835783   0.8459127 ]]

 [[ 1.13247866 -0.30656144]
  [ 1.73372891 -0.60812797]]

 [[ 1.90131787 -0.75540425]
  [ 1.827

[[[-0.42500653 -0.06640186]
  [ 0.26973282  0.76900181]]

 [[ 1.03373781  0.06989267]
  [ 0.47341096 -0.50938712]]

 [[ 1.9409587  -0.00977298]
  [ 0.6411104   2.72641211]]

 [[ 1.55841364 -0.49124681]
  [-0.3647093  -0.10657347]]

 [[ 1.27941591 -1.03836996]
  [-1.55136002 -1.32375146]]

 [[ 0.37251763 -0.45399082]
  [-0.75188213 -1.18064491]]

 [[ 0.79119816 -0.01239731]
  [-0.28846512 -0.17851604]]

 [[-0.30694219  0.00844952]
  [ 0.41970123  2.24621189]]

 [[-1.79637966 -0.32621064]
  [-0.20235876  0.22104347]]]
[[[-0.41586802 -0.02723422]
  [ 0.23690359  0.75986331]]

 [[ 1.02037177  0.09544376]
  [ 0.36048366 -0.49602108]]

 [[ 1.94504065  0.0510037 ]
  [ 0.53969517  2.72233016]]

 [[ 1.5599663  -0.43656445]
  [-0.42739142 -0.10812613]]

 [[ 1.2907979  -0.99013262]
  [-1.51662957 -1.33513344]]

 [[ 0.3748361  -0.44043937]
  [-0.73322769 -1.18296338]]

 [[ 0.78556775 -0.03808786]
  [-0.26101202 -0.17288563]]

 [[-0.29071995  0.05031056]
  [ 0.40603453  2.22998965]]

 [[-1.77413061

[[[-0.26059676  0.56040819]
  [-0.18607039  0.60459204]]

 [[ 0.76578523  0.34901835]
  [-1.3843576  -0.24143454]]

 [[ 1.99961251  0.89145212]
  [-0.92685669  2.6677583 ]]

 [[ 1.57617043  0.35752301]
  [-1.28801541 -0.12433026]]

 [[ 1.49622725 -0.16483462]
  [-0.8303539  -1.54056279]]

 [[ 0.41843817 -0.19684558]
  [-0.39748383 -1.22656545]]

 [[ 0.69108694 -0.41541677]
  [ 0.11148865 -0.07840482]]

 [[-0.00736405  0.7093538 ]
  [ 0.29771571  1.94663375]]

 [[-1.37790045  0.52436191]
  [ 0.3939555  -0.19743574]]]
[[[-0.25213261  0.59508274]
  [-0.20508299  0.5961279 ]]

 [[ 0.74937464  0.35469972]
  [-1.49009055 -0.22502395]]

 [[ 2.00097305  0.93604684]
  [-1.00763275  2.66639775]]

 [[ 1.57954568  0.40100836]
  [-1.33450644 -0.12770551]]

 [[ 1.51559734 -0.11193622]
  [-0.78280422 -1.55993289]]

 [[ 0.42442398 -0.18098898]
  [-0.376048   -1.23255126]]

 [[ 0.68644344 -0.43727381]
  [ 0.12967649 -0.07376132]]

 [[ 0.00792794  0.7508336 ]
  [ 0.29993944  1.93134176]]

 [[-1.35329165

[[[-0.22998786  0.95621171]
  [-0.15089472  0.57398314]]

 [[ 0.57184182  0.0494506 ]
  [-2.68406225 -0.04749113]]

 [[ 1.91539977  1.20640288]
  [-1.58745345  2.75197104]]

 [[ 1.76626046  0.69278018]
  [-1.67760513 -0.31442029]]

 [[ 2.09415927  0.54468762]
  [-0.10259364 -2.13849482]]

 [[ 0.67457232  0.01536637]
  [-0.13483093 -1.4826996 ]]

 [[ 0.70650089 -0.65223786]
  [ 0.15135053 -0.09381877]]

 [[ 0.02825815  1.29188368]
  [ 0.69419579  1.91101155]]

 [[-1.11130459  1.36699281]
  [ 1.33199575 -0.4640316 ]]]
[[[-2.36344066e-01  9.57097919e-01]
  [-1.17699657e-01  5.80339349e-01]]

 [[ 5.67318189e-01 -8.75101088e-04]
  [-2.70069505e+00 -4.29674999e-02]]

 [[ 1.90339926e+00  1.18052115e+00]
  [-1.55565741e+00  2.76397155e+00]]

 [[ 1.78357245e+00  6.73491536e-01]
  [-1.65742893e+00 -3.31732283e-01]]

 [[ 2.14070656e+00  5.64620735e-01]
  [-6.96335143e-02 -2.18504211e+00]]

 [[ 6.97155892e-01  2.18925002e-02]
  [-1.28744601e-01 -1.50528317e+00]]

 [[ 7.13738691e-01 -6.50819336e-01

[[[-0.35233698  0.47244739]
  [ 0.73946198  0.69633227]]

 [[ 0.55559084 -1.0611868 ]
  [-1.54827533 -0.03124015]]

 [[ 1.74051298  0.02493091]
  [ 0.11254059  2.92685783]]

 [[ 1.88060195 -0.23387723]
  [-0.50012839 -0.42876178]]

 [[ 2.45436696  0.38592938]
  [ 0.19271401 -2.49870251]]

 [[ 0.8682192   0.01023639]
  [-0.16633024 -1.67634648]]

 [[ 0.81434559 -0.29565877]
  [-0.4879962  -0.20166347]]

 [[-0.1864434   0.98744009]
  [ 1.55824149  2.1257131 ]]

 [[-1.19934588  1.3265484 ]
  [ 1.93337614 -0.37599031]]]
[[[-3.55563780e-01  4.15399064e-01]
  [ 7.91250902e-01  6.99559063e-01]]

 [[ 5.58982834e-01 -1.11308694e+00]
  [-1.39289197e+00 -3.46321445e-02]]

 [[ 1.73937094e+00 -7.21101734e-02]
  [ 2.59137994e-01  2.92799986e+00]]

 [[ 1.86577328e+00 -3.10519641e-01]
  [-3.94103533e-01 -4.13933108e-01]]

 [[ 2.42253516e+00  3.39483965e-01]
  [ 1.79958953e-01 -2.46687071e+00]]

 [[ 8.54653348e-01  6.74630001e-04]
  [-1.77648613e-01 -1.66278063e+00]]

 [[ 8.14792102e-01 -2.56628353e-01

...

[[[-274.41111572   86.7983379 ]
  [ 161.83871039  274.755111  ]]

 [[ 299.667654    -94.64363632]
  [-176.31596807 -299.14330331]]

 [[-215.17550662   68.76626309]
  [ 128.28549461  219.84287743]]

 [[ 458.98484728 -144.97986101]
  [-270.09111981 -457.53300711]]

 [[ 820.39904026 -259.5912997 ]
  [-483.72640285 -820.4433758 ]]

 [[ 455.33760817 -144.17749064]
  [-268.68443603 -456.14573545]]

 [[ 236.8197356   -74.77814423]
  [-139.41299716 -236.20705349]]

 [[-594.80334874  188.35954205]
  [ 351.11720144  596.74261844]]

 [[-390.74254633  123.22084671]
  [ 229.71750042  389.16721014]]]
[[[-274.69526471   86.89037672]
  [ 162.08964616  275.03926   ]]

 [[ 299.94317452  -94.88094979]
  [-176.5511782  -299.41882383]]

 [[-215.42160413   68.76449616]
  [ 128.4906911   220.08897494]]

 [[ 459.48110522 -145.17043583]
  [-270.30191473 -458.02926505]]

 [[ 821.33461167 -259.77801663]
  [-484.09396551 -821.37894721]]

 [[ 455.84710991 -144.29637436]
  [-268.92481654 -456.65523719]]

 [[ 237.06

[[[-278.86273305   88.19046502]
  [ 164.51067827  279.20672833]]

 [[ 304.50749489  -96.24955347]
  [-179.15288316 -303.9831442 ]]

 [[-218.71011166   69.82407016]
  [ 130.4351498   223.37748247]]

 [[ 466.41780038 -147.3676863 ]
  [-274.4114802  -464.96596021]]

 [[ 833.72426126 -263.77798925]
  [-491.51573433 -833.76859681]]

 [[ 462.73665048 -146.50458445]
  [-273.02711782 -463.54477776]]

 [[ 240.65578781  -75.97716768]
  [-141.70498738 -240.0431057 ]]

 [[-604.46041305  191.41286777]
  [ 356.86520597  606.39968275]]

 [[-397.05178193  125.24373673]
  [ 233.50440333  395.47644574]]]
[[[-279.14919854   88.30887237]
  [ 164.71594873  279.49319383]]

 [[ 304.79565098  -96.42384762]
  [-179.44540248 -304.27130029]]

 [[-218.95252937   69.88854019]
  [ 130.55453015  223.61990018]]

 [[ 466.92061986 -147.513685  ]
  [-274.69411098 -465.46877969]]

 [[ 834.66016821 -263.97124985]
  [-491.92314292 -834.70450376]]

 [[ 463.24746476 -146.63000412]
  [-273.27710428 -464.05559204]]

 [[ 240.90

[[[-283.65224731   89.70605404]
  [ 167.34310168  283.9962426 ]]

 [[ 309.72562357  -97.91698908]
  [-182.23855687 -309.20127288]]

 [[-222.50679558   71.01646552]
  [ 132.67878385  227.17416638]]

 [[ 474.41439748 -149.90002966]
  [-279.11634656 -472.96255732]]

 [[ 848.04668024 -268.29462327]
  [-499.93624162 -848.09101579]]

 [[ 470.69132493 -149.01542319]
  [-277.70877312 -471.49945221]]

 [[ 244.78200942  -77.28169107]
  [-144.1423704  -244.1693273 ]]

 [[-614.8524003   194.70482129]
  [ 363.00467628  616.79166999]]

 [[-403.84893304  127.40694919]
  [ 237.53331103  402.27359685]]]
[[[-283.94737632   89.8106469 ]
  [ 167.53865837  284.2913716 ]]

 [[ 310.03486626  -98.06067236]
  [-182.47585301 -309.51051557]]

 [[-222.74854335   71.08129352]
  [ 132.81704226  227.41591416]]

 [[ 474.91867983 -150.05719841]
  [-279.40116268 -473.46683966]]

 [[ 848.96773789 -268.53574991]
  [-500.40608876 -849.01207344]]

 [[ 471.19848247 -149.15784626]
  [-277.98227092 -472.00660975]]

 [[ 245.03

[[[-288.17615376   91.21867283]
  [ 169.89504198  288.52014904]]

 [[ 314.67616313  -99.33205927]
  [-185.3435269  -314.15181244]]

 [[-226.08170821   72.32763291]
  [ 134.54630806  230.74907902]]

 [[ 481.98236941 -152.15997366]
  [-283.75865751 -480.53052924]]

 [[ 861.57922602 -272.55073212]
  [-507.96349731 -861.62356157]]

 [[ 478.20788361 -151.39493667]
  [-282.14292812 -479.01601089]]

 [[ 248.6792523   -78.57408121]
  [-146.35720658 -248.06657018]]

 [[-624.67592178  197.87605822]
  [ 368.69898052  626.61519148]]

 [[-410.28093763  129.46906161]
  [ 241.26836775  408.70560144]]]
[[[-288.47959844   91.31354118]
  [ 170.07426571  288.82359372]]

 [[ 315.00725042  -99.43669339]
  [-185.53522847 -314.48289973]]

 [[-226.32192391   72.40232899]
  [ 134.69022474  230.98929471]]

 [[ 482.48829722 -152.32128191]
  [-284.05561033 -481.03645705]]

 [[ 862.48460119 -272.83889558]
  [-508.4990223  -862.52893674]]

 [[ 478.71095414 -151.55459813]
  [-282.44023447 -479.51908142]]

 [[ 248.94

[[[-292.42349713   92.54337765]
  [ 172.52617907  292.76749241]]

 [[ 319.26225125 -100.99772442]
  [-188.08377447 -318.73790056]]

 [[-229.47336264   73.26021972]
  [ 136.67809556  234.14073345]]

 [[ 489.09262184 -154.48283275]
  [-287.80689069 -487.64078167]]

 [[ 874.36884457 -276.47391925]
  [-515.27338435 -874.41318011]]

 [[ 485.30052421 -153.58803911]
  [-286.25237895 -486.10865149]]

 [[ 252.34413281  -79.70907818]
  [-148.59486193 -251.73145069]]

 [[-633.88854925  200.82010255]
  [ 374.29709559  635.82781895]]

 [[-416.26799487  131.48513441]
  [ 245.03451243  414.69265868]]]
[[[-292.73400323   92.61505843]
  [ 172.72087842  293.07799851]]

 [[ 319.60521442 -101.10960246]
  [-188.20532046 -319.08086373]]

 [[-229.71596916   73.30330235]
  [ 136.87921084  234.38333997]]

 [[ 489.59886433 -154.67532276]
  [-288.07197185 -488.14702416]]

 [[ 875.26668523 -276.79765062]
  [-515.84081109 -875.31102077]]

 [[ 485.80245031 -153.75855319]
  [-286.56477285 -486.61057759]]

 [[ 252.60

In [55]:
X

,X 1,X 2,X3,X4,X5,X6,X7,X8,X9
0,"[[-0.5024535079758764, -0.5223841303522911], [...","[[1.1329509367318993, -0.31192378895415884], [...","[[1.9009645314459942, -0.7646729273357331], [1...","[[1.5106185502789733, -1.1321928702968007], [0...","[[1.128512005154864, -1.507618830714462], [-1....","[[0.3273816796567281, -0.5735812756328065], [-...","[[0.8361085255439067, 0.2934415391436299], [-0...","[[-0.433037351039346, -0.4619691099845434], [0...","[[-1.9870603853371522, -0.8100187632498369], [..."
1.0,"[[-0.5023202313729456, -0.5210633209248025], [...","[[1.1328343776101242, -0.31057957775176004], [...","[[1.901052667907002, -0.7623520337010367], [1....","[[1.5108545874937427, -1.1303309679203226], [0...","[[1.1290233394561318, -1.5065346260657846], [-...","[[0.3275766547288663, -0.5733491973627788], [-...","[[0.8360444107881996, 0.292535758490194], [-0....","[[-0.4328693578859022, -0.46065251353693976], ...","[[-1.9867161578374424, -0.8088073005002346], [..."
2.0,"[[-0.5019174120107182, -0.5171065623549493], [...","[[1.13247866243188, -0.30656144182817274], [1....","[[1.9013178704745037, -0.7554042453575445], [1...","[[1.5115577797895117, -1.1247532799352278], [0...","[[1.1305518890057111, -1.5032762440299245], [-...","[[0.3281580752323905, -0.5726496123224697], [-...","[[0.8358497385906254, 0.28982305250230594], [-...","[[-0.4323583884864249, -0.45670673203922946], ...","[[-1.9856769633891367, -0.8051710885729203], [..."
3.0,"[[-0.5012361892077088, -0.5105308611660175], [...","[[1.1318658667963082, -0.29991264517424643], [...","[[1.9017624775126551, -0.743874106639558], [1....","[[1.5127134941436409, -1.115483871206294], [0....","[[1.1330814171376984, -1.4978266728717489], [-...","[[0.32911551870444455, -0.5714725932051447], [...","[[0.8355176056346788, 0.28531730653583803], [-...","[[-0.4314837067452192, -0.45014382607875164], ...","[[-1.983923498401649, -0.7991048408320949], [-..."
4.0,"[[-0.5002621522936159, -0.501364549933778], [0...","[[1.1309667423475709, -0.2907045325462563], [1...","[[1.9023902502420569, -0.7278354111262508], [1...","[[1.5142977564738027, -1.1025628700544712], [0...","[[1.1365852719835021, -1.4901585259654826], [-...","[[0.3304319181128432, -0.5698020025374658], [-...","[[0.835036772612807, 0.279041586095671], [-0.6...","[[-0.4302115257968411, -0.440984016486214], [0...","[[-1.981424358427955, -0.7906003543242824], [-..."
...,...,...,...,...,...,...,...,...,...
1995.0,"[[-291.50398930648566, 92.30690489935833], [17...","[[318.2695343947137, -100.59305282999713], [-1...","[[-228.74031828513574, 73.14314065141477], [13...","[[487.5708266481485, -153.91500633822938], [-2...","[[871.6359123963091, -275.587027045898], [-513...","[[483.78141449399754, -153.10645166747184], [-...","[[251.55274119007976, -79.49794485620433], [-1...","[[-631.8985508418132, 200.2357449150059], [373...","[[-414.9651564121096, 131.0982498159269], [244..."
1996.0,"[[-291.8085387706515, 92.39182792402309], [172...","[[318.59563076340964, -100.73248905076859], [-...","[[-228.98492790619744, 73.18697692569886], [13...","[[488.0788391367505, -154.0979904502135], [-28...","[[872.5525222120125, -275.86949166166386], [-5...","[[484.28964826076265, -153.26268901832063], [-...","[[251.81555121171118, -79.57184725120403], [-1...","[[-632.558155227884, 200.43920933666212], [373...","[[-415.3929454344049, 131.24057582063097], [24..."
1997.0,"[[-292.1149546189577, 92.47015820091168], [172...","[[318.92599040676424, -100.86961204051046], [-...","[[-229.22947093791856, 73.22418939237551], [13...","[[488.58607601023897, -154.2883917690824], [-2...","[[873.4640223581089, -276.1643358262397], [-51...","[[484.79620977257605, -153.42284194299495], [-...","[[252.0793260443438, -79.64178724696549], [-14...","[[-633.221244571067, 200.63393574188296], [373...","[[-415.8266925945651, 131.37031155707567], [24..."
1998.0,"[[-292.4234971252166, 92.54337764782473], [172...","[[319.26225125114087, -100.99772441889759], [-...","[[-229.4733626426105, 73.2602197227

### Set display of pandas dataframes and Series to 11 decimal places

In [30]:
pd.set_option("display.precision", 11)

In [56]:
X.to_pickle(path+"X_test9.pkl")
V.to_pickle(path+"V_test9.pkl")

# Lyapunov exponent

1. Thermalise system
1. Perturb system
1. Evolve both original and perturbed system, measuring the exponential rate at which they diverge.

In [ ]:
def simulation_2_Points(dim, N, delta_t):
    # N is the length of the side of the individual position square matrices.
    # dim is the number of position coordinates we need.
    
    # Create position coordinate matrices with gaussian distributed elements.
    
    X_list = np.random.normal(0, 1, size = (dim, N, N))
    
    # The velocity Verlet algorithm conserves Gauss' law, so if it starts off true, it remains true. 
    # So just ensure the initial V^i are zero matrices.  

    V_initial = np.zeros((N, N))
    i = 0
    V_list = []
    while i < len(X_list):
        V_list.append(V_initial)
        i += 1
    
    # Use the scalar equation of motion in our gauge to calculate the initial accelerations from the initial positions
    
    A_list = acceleration(X_list)

    # Define number of steps between measurement recordings
    record_steps = 100000
    
    # How many iterations the simulation will go on for after it has been thermalised.
    simulation_repetitions = 10000000
    
    
    # Thermalise the system. , i.e. evolve it for long enough time that it presumably reaches a "typical configuration".
    # Here Swapno suggests to evolve it, starting anywhere, for 1000 seconds, dt = 10e-5 s.
    thermalised_X_array, thermalised_V_array, thermalised_A_array = thermalising_algorithm(X_list, V_list, A_list, delta_t)
        
    
    #### New Code ####
    
    # Perturb the thermalised_initial coordinates to get the second system
    
    
    
        
    ####          #### 
    
    # Labels for coordinates dictionary
    X_labels = []
    j = 1
    while j <= dim:
        X_labels.append(f"X {j}")
        
        j+=1
    
    V_labels = []
    j = 1
    while j <= dim:
        V_labels.append(f"V {j}")
        
        j+=1
    
    
    # Define the times after thermalisation that the coordinates were recorded at.
    times = np.arange(0, simulation_repetitions*delta_t + delta_t*record_steps, delta_t*record_steps)
    
    # Make dataframes out of the dictionaries
    
    # Check if H is conserved

    X1 = pd.Series(recorded_X1).T
    X2 = pd.Series(recorded_X2).T
    V1 = pd.Series(recorded_V1).T
    V2 = pd.Series(recorded_V2).T

    X = pd.concat([X1, X2], axis = 1)
    X = X.rename(columns = {0:"X 1", 1:"X 2"})
    
    V = pd.concat([V1, V2], axis = 1)
    V = V.rename(columns = {0:"V 1", 1:"V 2"})
    

    
    return X, V